In [1]:
import numpy as np
import polars as pl

import time
import math
import warnings
warnings.filterwarnings('ignore')

In [2]:
# load datasets
dataset = pl.read_csv('../data/competition/Train.csv', ignore_errors=True)
sbmssn = pl.read_csv('../data/competition/Test.csv', ignore_errors=True)

In [3]:
dataset.head(n=5)

child_id,data_year,child_date,child_age,child_enrolment_date,child_months_enrolment,child_grant,child_years_in_programme,child_height,child_observe_attentive,child_observe_concentrated,child_observe_diligent,child_observe_interested,child_observe_total,child_gender,child_dob,child_zha,child_stunted,child_attends,child_attendance,child_languages,child_age_group,id_mn_best,prov_best,id_dc_best,dc_best,mn_best,ward_best,id_enumerator,id_facility,pqa_date,pqa_class_age,pqa_class_age_1,pqa_class_age_2,pqa_class_age_3,pqa_class_age_4,pqa_class_age_5,…,positionother,positionotherreason,sef_ind,language_match,elp_ind,gps_ind,pre_covid,ses_proxy,quintile_used,id_facility_n,id_ward_n,id_mn_n,id_dc_n,id_prov_n,language_assessment_w2,ses_cat,obs_lighting_1,obs_lighting_2,obs_lighting_3,obs_lighting_4,obs_lighting_5,obs_lighting_6,obs_lighting_8,obs_cooking_1,obs_cooking_2,obs_cooking_3,obs_cooking_4,obs_cooking_5,obs_cooking_6,obs_heating_1,obs_heating_2,obs_heating_3,obs_heating_4,obs_heating_5,obs_heating_6,obs_heating_7,target
str,f64,str,f64,str,f64,str,str,f64,str,str,str,str,f64,str,str,f64,str,str,f64,str,str,str,str,str,str,str,f64,f64,f64,str,str,str,str,str,str,str,…,str,str,str,f64,str,str,str,f64,str,f64,f64,f64,f64,f64,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""ID_SYSJ2FM0D""",2022.0,"""2022-02-03""",59.0,null,null,null,null,null,"""Sometimes""","""Sometimes""","""Sometimes""","""Sometimes""",4.0,"""Female""","""2017-02-06""",null,null,null,null,null,"""50-59 months""","""GT421""","""GAUTENG""","""DC42""","""SEDIBENG""","""EMFULENI""",39.0,20005.0,761.0,null,null,null,null,null,null,null,…,null,null,"""Yes""",1.0,"""No""","""Yes""","""Post COVID""",2.0,"""Yes""",7.0,14.0,107.0,134.0,1051.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,51.5
"""ID_J5BTFOZR3""",2019.0,null,60.163933,null,null,null,"""1st year in th…",103.0,"""Sometimes""","""Almost never""","""Sometimes""","""Often""",4.0,"""Female""",null,-1.356791,"""Normal""",null,null,null,"""60-69 months""",null,"""KWAZULU-NATAL""","""DC22""","""UMGUNGUNDLOVU""",null,null,null,458.0,null,null,null,null,null,null,null,…,null,null,"""Yes""",1.0,"""No""","""No""","""Pre-COVID""",4.0,"""Yes""",24.0,null,null,367.0,1832.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,55.869999
"""ID_R00SN7AUD""",2022.0,"""2022-03-11""",69.0,null,null,null,null,108.400002,"""Often""","""Often""","""Sometimes""","""Often""",7.0,"""Male""","""2016-05-24""",-1.250863,"""Normal""",null,null,null,"""60-69 months""","""CPT""","""WESTERN CAPE""","""CPT""","""CITY OF CAPE T…","""CITY OF CAPE T…",85.0,20001.0,925.0,null,null,null,null,null,null,null,…,null,null,"""No""",1.0,"""Yes""","""Yes""","""Post COVID""",1.0,"""No""",8.0,24.0,1448.0,1448.0,3214.0,null,"""R0-110""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,47.52
"""ID_BSSK60PAZ""",2021.0,"""2021-10-13""",53.0,"""2020-01-15""",20.0,"""No""","""1st year in th…",98.099998,"""Almost always""","""Almost always""","""Sometimes""","""Often""",9.0,"""Male""","""2017-05-08""",-1.830364,"""Normal""",null,null,null,"""50-59 months""","""WC025""","""WESTERN CAPE""","""DC2""","""CAPE WINELANDS…","""BREEDE VALLEY""",18.0,2689.0,308.0,null,null,null,null,null,null,null,…,null,null,"""Yes""",1.0,"""Yes""","""Yes""","""Post COVID""",3.0,"""No""",4.0,22.0,76.0,629.0,3214.0,null,"""R291-750""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,58.599998
"""ID_IZTY6TC4D""",2021.0,"""2021-10-13""",57.0,"""2021-10-13""",0.0,null,"""2nd year in pr…",114.0,"""Almost always""","""Almost always""","""Almost always""","""Almost always""",12.0,"""Female""","""2016-12-19""",1.3292638,"""Normal""",null,null,null,"""50-59 months""","""KZN293""","""KWAZULU-NATAL""","""DC29""","""ILEMBE""","""NDWEDWE"

In [4]:
# drop some columns
deselect_col = [
    'child_id',
    'obs_materials', 'obs_handwashing', 'obs_area', 'obs_toilet',
    'obs_equipment', 'obs_safety', 'obs_hazard', 'pqa_class_age',
    'pra_groupings', 'pra_plans', 'obs_access_disability', 'pra_cohort',
    'pra_plan_4yrs', 'pra_plan_5yrs', 'pra_plan_4yrsother', 'pra_plan_5yrsother',
    'pra_qualification', 'pra_ncf_trainer', 'pra_training', 'pri_language',
    'pri_meal', 'pri_money',  'pri_funding_salary', 'pri_expenseother',
    'pri_clinic_travel', 'pri_covid_awareness', 'pri_covid_precautions',
    'pri_food_type', 'pri_records', 'pri_support_provider', 
    'pri_qualification', 'pri_email_network_forum', 
    'pri_reason_register_year',  
     
    #'pri_support_providerother', 'pri_languageother', 'pri_moneyother',
    #'pri_dsd_unregistered_other', 'pri_dsd_conditional_other', 'pri_landother', 'pri_fundingother', 'pri_facilitiesother',
    #'pri_fees_exceptions_other', 'pri_staff_changes_reasonsother', 'hle_ecd_other', 'obs_water_running_none', 'obs_waterother'
]

dataset = dataset.select(pl.all().exclude(deselect_col))
sbmssn = sbmssn.select(pl.all().exclude(list(set(deselect_col) - set(['child_id']))))

In [5]:
# date columns we are going to recode
date_col = ['child_date', 'child_enrolment_date', 'child_dob', 'pqa_date', 'pra_date', 'pri_date', 'obs_date']

for cn in date_col: 
    dataset = dataset.with_columns(pl.col(cn).str.strptime(pl.Date, '%Y-%m-%d'))
    sbmssn = sbmssn.with_columns(pl.col(cn).str.strptime(pl.Date, '%Y-%m-%d'))

for cn in date_col: 
    dataset = dataset.with_columns([
        pl.col(cn).dt.year().alias(cn + '_year'), 
        pl.col(cn).dt.month().alias(cn + '_month'), 
        pl.col(cn).dt.quarter().alias(cn + '_quarter'),
        pl.col(cn).dt.week().alias(cn + '_week'), 
        pl.col(cn).cast(pl.Int32)
    ])

    sbmssn = sbmssn.with_columns([
        pl.col(cn).dt.year().alias(cn + '_year'), 
        pl.col(cn).dt.month().alias(cn + '_month'), 
        pl.col(cn).dt.quarter().alias(cn + '_quarter'),
        pl.col(cn).dt.week().alias(cn + '_week'), 
        pl.col(cn).cast(pl.Int32)
    ])

In [6]:
# Convert this to categorical
dataset = dataset.with_columns([
    pl.col('language_match').map_dict({0: 'No', 1: 'Yes'}, default=None).cast(pl.Categorical)
])

sbmssn = sbmssn.with_columns([
    pl.col('language_match').map_dict({0: 'No', 1: 'Yes'}, default=None).cast(pl.Categorical)
])

In [7]:
print(dataset['child_languages'].unique().to_list(), sep='\n')

['Afrikaans', None, 'Tshivenda', 'English + Setswana +', 'isiZulu + isiXhosa +', 'English + isiNdebele +', 'isiZulu', 'English + isiZulu +', 'English + Xitsonga +', 'Sesotho se Leboa (Sepedi)', 'English + isiZulu + isiXhosa', 'Afrikaans + Setswana +', 'isiXhosa', 'Setswana', 'English + isiXhosa +', 'English', 'isiXhosa + Xitsonga +', 'English + Afrikaans +', 'English + Sesotho se Leboa (Sepedi) +', 'Sesotho']


In [8]:
# extract child languages here
child_langs = ['Afrikaans', 'English', 'isiNdebele', 'isiXhosa',
               'isiZulu', 'Xitsonga', 'Setswana', 'Sesotho', 'Sepedi']

for ln in child_langs:
    dataset = dataset.with_columns([
        pl.col('child_languages').
        str.contains(ln).
        map_dict({0: 'No', 1: 'Yes'}, default=None).
        cast(pl.Categorical).
        alias(f'child_languages_{ln}')
    ])

    sbmssn = sbmssn.with_columns([
        pl.col('child_languages').
        str.contains(ln).
        map_dict({0: 'No', 1: 'Yes'}, default=None).
        cast(pl.Categorical).
        alias(f'child_languages_{ln}')
    ])

dataset = dataset.drop('child_languages')
sbmssn = sbmssn.drop('child_languages')

In [9]:
print(dataset['pri_languages'].unique().to_list(), sep='\n')

['Setswana', 'Afrikaans + isiZulu +', 'isiZulu', 'English + isiZulu +', None, 'isiXhosa', 'English + isiXhosa +', 'Sesotho', 'English + Afrikaans + isiXhosa', 'English + Afrikaans +', 'English', 'Afrikaans']


In [10]:
# extract pri_languages here
pri_langs = ['Afrikaans', 'English', 'isiXhosa', 'isiZulu', 'Setswana', 'Sesotho']

for ln in pri_langs:
    dataset = dataset.with_columns([
        pl.col('pri_languages').
        str.contains(ln).
        map_dict({0: 'No', 1: 'Yes'}, default=None).
        cast(pl.Categorical).
        alias(f'pri_languages_{ln}')
    ])

    sbmssn = sbmssn.with_columns([
        pl.col('pri_languages').
        str.contains(ln).
        map_dict({0: 'No', 1: 'Yes'}, default=None).
        cast(pl.Categorical).
        alias(f'pri_languages_{ln}')
    ])

dataset = dataset.drop('pri_languages')
sbmssn = sbmssn.drop('pri_languages')

In [11]:
# fix the gps column and add polar coordinates
dataset = dataset.with_columns(
    pl.col('gps').str.split(' ').\
    arr.to_struct(n_field_strategy='max_width').\
    struct.rename_fields(['gps_lat', 'gps_lon'])).\
    unnest('gps')

sbmssn = sbmssn.with_columns(
    pl.col('gps').str.split(' ').\
    arr.to_struct(n_field_strategy='max_width').\
    struct.rename_fields(['gps_lat', 'gps_lon'])).\
    unnest('gps')

# cast to numeric
for cn in ['gps_lat', 'gps_lon']:
    dataset = dataset.with_columns(pl.col(cn).cast(pl.Float64))
    sbmssn = sbmssn.with_columns(pl.col(cn).cast(pl.Float64))

In [12]:
# numerics I code as categories
numcat_columns = ['id_ward', 'id_enumerator', 'id_facility', 'id_team']

for cn in numcat_columns: 
    dataset = dataset.with_columns(pl.col(cn).cast(pl.Int64).cast(pl.Utf8).cast(pl.Categorical))
    sbmssn = sbmssn.with_columns(pl.col(cn).cast(pl.Int64).cast(pl.Utf8).cast(pl.Categorical))

In [13]:
# replace values with NA's in train.data
dataset = dataset.with_columns([
    pl.when(pl.col(pl.Utf8).str.contains('Do Not Know|Missing|Don\'t know|Refuse|DON\'T KNOW|DON\'T KNOW.|DON\'T KNOW THE REASON FOR CONDITIONAL'))
    .then(None)
    .otherwise(pl.col(pl.Utf8))
    .keep_name()
])

sbmssn = sbmssn.with_columns([
    pl.when(pl.col(pl.Utf8).str.contains('Do Not Know|Missing|Don\'t know|Refuse|DON\'T KNOW|DON\'T KNOW.|DON\'T KNOW THE REASON FOR CONDITIONAL'))
    .then(None)
    .otherwise(pl.col(pl.Utf8))
    .keep_name()
])

In [14]:
# check where dataset and sbmssn diasgree on data types
for cn in dataset.select(pl.exclude('target')).columns:
    if dataset[cn].dtype != sbmssn[cn].dtype:
        print(f'Column {cn} has different data types between the two data frames.')

Column pri_network_type has different data types between the two data frames.
Column pri_staff_changes_reasons has different data types between the two data frames.
Column pri_food_donor has different data types between the two data frames.


In [15]:
# fix those columns
for answer in [1, 2, 3, 97]:
    dataset = dataset.with_columns([
        pl.col('pri_network_type').
        str.contains(ln).
        map_dict({0: 'No', 1: 'Yes'}, default=None).
        cast(pl.Categorical).
        alias(f'pri_network_type_{answer}')
    ])

    sbmssn = sbmssn.with_columns([
        pl.col('pri_network_type').
        str.contains(ln).
        map_dict({0: 'No', 1: 'Yes'}, default=None).
        cast(pl.Categorical).
        alias(f'pri_network_type_{answer}')
    ])

for answer in [1, 2, 3, 4, 5, 6, 97]:
    dataset = dataset.with_columns([
        pl.col('pri_staff_changes_reasons').
        str.contains(ln).
        map_dict({0: 'No', 1: 'Yes'}, default=None).
        cast(pl.Categorical).
        alias(f'pri_staff_changes_reasons_{answer}')
    ])

    sbmssn = sbmssn.with_columns([
        pl.col('pri_staff_changes_reasons').
        str.contains(ln).
        map_dict({0: 'No', 1: 'Yes'}, default=None).
        cast(pl.Categorical).
        alias(f'pri_staff_changes_reasons_{answer}')
    ])

for answer in [1, 2, 3, 4, 97]:
    dataset = dataset.with_columns([
        pl.col('pri_food_donor').
        str.contains(ln).
        map_dict({0: 'No', 1: 'Yes'}, default=None).
        cast(pl.Categorical).
        alias(f'pri_food_donor_{answer}')
    ])

    sbmssn = sbmssn.with_columns([
        pl.col('pri_food_donor').
        str.contains(ln).
        map_dict({0: 'No', 1: 'Yes'}, default=None).
        cast(pl.Categorical).
        alias(f'pri_food_donor_{answer}')
    ])

# drop the splitted columns
dataset = dataset.drop(
    ['pri_network_type', 'pri_staff_changes_reasons', 'pri_food_donor'])
sbmssn = sbmssn.drop(
    ['pri_network_type', 'pri_staff_changes_reasons', 'pri_food_donor'])

In [16]:
# categorical encoding of some other survey responses
for answer in [1, 2, 3, 4, 5, 97]:
    dataset = dataset.with_columns([
        pl.col('health').
        str.contains(ln).
        map_dict({0: 'No', 1: 'Yes'}, default=None).
        cast(pl.Categorical).
        alias(f'health_{answer}')
    ])

    sbmssn = sbmssn.with_columns([
        pl.col('health').
        str.contains(ln).
        map_dict({0: 'No', 1: 'Yes'}, default=None).
        cast(pl.Categorical).
        alias(f'health_{answer}')
    ])

# drop the splitted columns
dataset = dataset.drop('health')
sbmssn = sbmssn.drop('health')

In [17]:
# other strings to categories
for cn in dataset.select(pl.col(pl.Utf8)).columns: 
    dataset = dataset.with_columns(pl.col(cn).cast(pl.Categorical))
    sbmssn = sbmssn.with_columns(pl.col(cn).cast(pl.Categorical))

In [18]:
# save as parquet for use in tuning and sensitivity
dataset.write_parquet("../data/clean/dataset.parquet")
sbmssn.write_parquet("../data/clean/sbmssn.parquet")